Use this script to begin the fine tuning process. Feel free to swap out the sample data sets with your own data. 

In [ ]:
# REQUIRED PACKAGES - PLEASE INSTALL BEFORE RUNNING THE CODE!
%%capture
%pip install -r requirements.txt

In [ ]:
# Import the required packages
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = "fine_tuned"

In [ ]:
#Load training dataset
dataset = load_dataset("text", data_files={"data/jarjar_sample_data.txt"}) # insert data for the fine-tuning process
print(dataset.column_names)
print(dataset)
dataset["train"].features
print(dataset["train"])
print(dataset["train"]["text"])

In [ ]:
# Setup quantization
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer(transfer text into tokens)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Load LoRA configuration
peft_args = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
# Train model
trainer.train()

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
# Merge trained model with base model

from peft import PeftConfig, PeftModel 
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "NousResearch/Llama-2-7b-chat-hf" #path/to/your/model/or/name/on/hub" 
adapter_model_name = new_model

model2 = AutoModelForCausalLM.from_pretrained(base_model_name) 
model2 = PeftModel.from_pretrained(model2, adapter_model_name)

tokenizer2 = AutoTokenizer.from_pretrained(base_model_name)

model2 = model2.merge_and_unload() 
tokenizer2.save_pretrained("merged") 
model2.generation_config.temperature=None
model2.generation_config.top_p=None
model2.save_pretrained("merged")